## Imports and DB connection

In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import joblib
from sqlalchemy import create_engine
import psycopg2

In [13]:
# load env variables
load_dotenv()
DATABASE_URL = os.getenv("ML_DATABASE_URL")

# connect to postgres db with sqlalchemy engine
engine = create_engine(DATABASE_URL)

# get player stats
query = """
SELECT pg.player_id, pg.game_id, pg.game_date, pg.matchup, p.team_abbreviation,
       pg.minutes, pg.points, pg.assists, pg.rebounds, pg.steals, pg.blocks, pg.turnovers
FROM player_game_stats pg
JOIN players p ON pg.player_id = p.id
"""
df_raw = pd.read_sql(query, engine)
df_raw.head(2)


,player_id,game_id,game_date,matchup,team_abbreviation,minutes,points,assists,rebounds,steals,blocks,turnovers
0,1630173,0022500307,2025-11-30,SAC vs. MEM,SAC,24.0,11.0,0.0,7.0,0.0,0.0,0.0
1,1630173,0022500077,2025-11-28,SAC @ UTA,SAC,19.0,2.0,1.0,6.0,0.0,2.0,0.0


## Feature engineering
1. Adding rolling average of basic stats 

In [14]:
# copy for feature engineering
df_features = df_raw.copy()

# adding indicator for home/away based on @/vs symbols
df_features["is_home"] = df_features["matchup"].apply(
    lambda x: 1 if "@" not in x else 0
)

# sort by player and game date
df_features = df_features.sort_values(["player_id", "game_date"])

# calcultae rolling averages for the base stats in last 5 games
df_features["avg_points_last5"] = df_features.groupby("player_id")["points"].transform(
    lambda x: x.rolling(5, min_periods=1).mean().shift(1)
)
df_features["avg_assists_last5"] = df_features.groupby("player_id")[
    "assists"
].transform(lambda x: x.rolling(5, min_periods=1).mean().shift(1))
df_features["avg_rebounds_last5"] = df_features.groupby("player_id")[
    "rebounds"
].transform(lambda x: x.rolling(5, min_periods=1).mean().shift(1))


In [15]:
df_features.head(5)

,player_id,game_id,game_date,matchup,team_abbreviation,minutes,points,assists,rebounds,steals,blocks,turnovers,is_home,avg_points_last5,avg_assists_last5,avg_rebounds_last5
1026,2544,0022500253,2025-11-18,LAL vs. UTA,LAL,30.0,11.0,12.0,3.0,1.0,0.0,1.0,1,NaN,NaN,NaN
1025,2544,0022500282,2025-11-23,LAL @ UTA,LAL,34.0,17.0,8.0,6.0,1.0,0.0,2.0,0,11.000000,12.000000,3.0
1024,2544,0022500059,2025-11-25,LAL vs. LAC,LAL,32.0,25.0,6.0,6.0,1.0,1.0,3.0,1,14.000000,10.000000,4.5
1023,2544,0022500078,2025-11-28,LAL vs. DAL,LAL,34.0,13.0,7.0,5.0,1.0,0.0,2.0,1,17.666667,8.666667,5.0
1027,2544,0022500586,2026-01-15,LAL vs. CHA,LAL,33.0,29.0,6.0,9.0,2.0,1.0,3.0,1,16.500000,8.250000,5.0


2. Rolling average of teamamte influence (taking only teammate assists for now).

In [16]:
# average assists of other teammates in last 5 games
teammate_avg_assists = []
for idx, row in df_features.iterrows():
    team = row["team_abbreviation"]
    game_date = row["game_date"]
    player_id = row["player_id"]
    # get teammates last 5 games excluding current player
    teammates = df_features[
        (df_features["team_abbreviation"] == team)
        & (df_features["player_id"] != player_id)
        & (df_features["game_date"] < game_date)
    ]
    last5 = teammates.groupby("player_id").tail(5)
    avg_assist = last5["assists"].mean() if not last5.empty else 0
    teammate_avg_assists.append(avg_assist)
df_features["teammate_avg_assists_last5"] = teammate_avg_assists

df_features.head(6)

,player_id,game_id,game_date,matchup,team_abbreviation,minutes,points,assists,rebounds,steals,blocks,turnovers,is_home,avg_points_last5,avg_assists_last5,avg_rebounds_last5,teammate_avg_assists_last5
1026,2544,0022500253,2025-11-18,LAL vs. UTA,LAL,30.0,11.0,12.0,3.0,1.0,0.0,1.0,1,NaN,NaN,NaN,2.193548
1025,2544,0022500282,2025-11-23,LAL @ UTA,LAL,34.0,17.0,8.0,6.0,1.0,0.0,2.0,0,11.000000,12.000000,3.0,2.121212
1024,2544,0022500059,2025-11-25,LAL vs. LAC,LAL,32.0,25.0,6.0,6.0,1.0,1.0,3.0,1,14.000000,10.000000,4.5,2.058824
1023,2544,0022500078,2025-11-28,LAL vs. DAL,LAL,34.0,13.0,7.0,5.0,1.0,0.0,2.0,1,17.666667,8.666667,5.0,1.972222
1027,2544,0022500586,2026-01-15,LAL vs. CHA,LAL,33.0,29.0,6.0,9.0,2.0,1.0,3.0,1,16.500000,8.250000,5.0,2.090909
1304,200768,0022500149,2025-11-02,PHI @ BKN,PHI,3.0,3.0,1.0,0.0,1.0,0.0,0.0,0,NaN,NaN,NaN,3.131579


3. Opponent strength metrics. 

* Taking points allowed, blocks, steals, turnovers data for now 

In [ ]:
# opponent strength (points allowed, blocks, steals, turnovers)
opponent_avg_points_allowed = []
opponent_avg_blocks = []
opponent_avg_steals = []
opponent_avg_turnovers = []

for idx, row in df_features.iterrows():
    matchup = row["matchup"]
    player_team = row["team_abbreviation"]
    if " vs. " in matchup:
        opponent_team = matchup.split(" vs. ")[1]
    else:
        opponent_team = matchup.split(" @ ")[1]

    opp_games = df_features[
        (df_features["team_abbreviation"] == opponent_team)
        & (df_features["game_date"] < row["game_date"])
    ]
    last5 = opp_games.tail(5)

    opponent_avg_points_allowed.append(last5["points"].mean() if not last5.empty else 0)
    opponent_avg_blocks.append(last5["blocks"].mean() if not last5.empty else 0)
    opponent_avg_steals.append(last5["steals"].mean() if not last5.empty else 0)
    opponent_avg_turnovers.append(last5["turnovers"].mean() if not last5.empty else 0)

df_features["opponent_avg_points_allowed_last5"] = opponent_avg_points_allowed
df_features["opponent_avg_blocks_last5"] = opponent_avg_blocks
df_features["opponent_avg_steals_last5"] = opponent_avg_steals
df_features["opponent_avg_turnovers_last5"] = opponent_avg_turnovers
